In [1]:
R = [[(1,1),(1,2),(1,3)],
    [(1,4),(2,1),(2,2),(2,3),(2,4)]]

In [8]:
def row_regions(N,M):
    return [[(i,j) for j in range(M)] for i in range(N)]

def column_regions(N,M):
    return [[(i,j) for i in range(N)] for j in range(M)]

def region_lists(grid: np.array):
    """Lists of region indices from grid"""
    regions = defaultdict(list)
    for (i, j), r in np.ndenumerate(grid):
        regions[r].append((i, j))
    return list(regions.values())

In [19]:
from adict import adict

In [28]:
Q = adict(int)

In [29]:
def x(indices):
    """Variable label"""
    i, j = indices
    return f"x{i}_{j}"

In [30]:
def region_constraint(Q, r, k):
    for (i,j) in r:
        Q[ ( x((i,j)) , x((i,j)) ) ] += (-k+1)
        for (l,p) in r:
            if (l,p)!=(i,j):
                Q[ ( x((i,j)) , x((l,p)) ) ] += 1

In [33]:
Q = adict(int)
region_constraint(Q,R[0],2)
Q

adict(int,
      {('x1_1', 'x1_1'): -1,
       ('x1_1', 'x1_2'): 1,
       ('x1_1', 'x1_3'): 1,
       ('x1_2', 'x1_2'): -1,
       ('x1_2', 'x1_1'): 1,
       ('x1_2', 'x1_3'): 1,
       ('x1_3', 'x1_3'): -1,
       ('x1_3', 'x1_1'): 1,
       ('x1_3', 'x1_2'): 1})

In [34]:
Q = adict(int)
for r in R:
    region_constraint(Q,R[0],2)
for line in L:
    region_constraint(Q,L[0],2)
for row in R:
    region_constraint(Q,R[0],2)

Q

adict(int,
      {('x1_1', 'x1_1'): -2,
       ('x1_1', 'x1_2'): 2,
       ('x1_1', 'x1_3'): 2,
       ('x1_2', 'x1_2'): -2,
       ('x1_2', 'x1_1'): 2,
       ('x1_2', 'x1_3'): 2,
       ('x1_3', 'x1_3'): -2,
       ('x1_3', 'x1_1'): 2,
       ('x1_3', 'x1_2'): 2})

In [10]:
create_columns(3,4)

[[(0, 0), (1, 0), (2, 0)],
 [(0, 1), (1, 1), (2, 1)],
 [(0, 2), (1, 2), (2, 2)],
 [(0, 3), (1, 3), (2, 3)]]

In [13]:
for (x,y) in R[0]:
    print(x)

1
1
1


In [1]:
import neal

In [ ]:
sampler = neal.SimulatedAnnealingSampler()


In [ ]:
import ...

grid = 

Q = 

sample_set = sampler.sample_qubo(Q)

In [ ]:
import matplotlib.pyplot as plt

def draw_puzzle(grid,star):
    fig, ax = plt.subplots()
    mat = ax.imshow(grid, cmap='GnBu', interpolation='nearest')
    for x in range(star.shape[0]):
        for y in range(star.shape[1]):
            if star[x,y] == 1:
                ax.annotate("*", xy=(y, x), horizontalalignment='center', verticalalignment='center',size=30)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    
def sample_to_plot(grid,sample):
    star_positions = [ind(var) for var, value in sample.items() if value == 1]
    solution = np.zeros((N, N), dtype=int)
    for (i, j) in star_positions:
        solution[i, j] = 1
    draw_puzzle(grid,solution)

In [ ]:
# Try Sampling

sampleset = sampler.sample_qubo(Q, num_reads=20)
print(sampleset)

for sample in sampleset:
    print(confirm_solution(sample, regions, nstars))
    
for sample in sampleset:
    sample_to_plot(grid, sample)

In [ ]:
# Go to DWave

from dwave.system import LeapHybridSampler
LEAPsampler =   LeapHybridSampler()

# Try Sampling
LEAPsampleset = LEAPsampler.sample_qubo(Q)
print(LEAPsampleset)

In [ ]:
def find_neighbours(cell, region):
    neighbours = []
    for other_cell in region:
        manhattan = abs(cell[0] - other_cell[0]) + abs(cell[1] - other_cell[1])
        if manhattan == 1:
            neighbours.append(list(other_cell))
    return neighbours
        
def connected_component(region):
    conn = [list(region[0])] # first cell of connected component
    cell_idx = 0
    while cell_idx < len(conn): 
        cell = conn[cell_idx] 
        neighbours = find_neighbours(cell, region) # find neighbours of cell in region
        for neighbour in neighbours:
            if not neighbour in conn:
                conn.append(neighbour) # add found neighbours to connected component
        cell_idx += 1
    return conn

def connected(region):
    conn = connected_component(region)  
    conn_set = {tuple(pos) for pos in conn}
    region_set = {tuple(pos) for pos in region}
    return conn_set == region_set

In [ ]:
def confirm_solution_regions(sample_star, grid, nstars):
    '''Check if :grid: serves :sample_star: solution.'''
    
    regions = region_lists(grid) # do not check list/column regions 
    for region in regions:
        if not region_criterion(sample_star, region, nstars):
            return False   
        if not connected(region):
            return False
        
        
    return True

In [ ]:
'''Confirm Solution of Regions'''

# sampleset = LEAPsampleset
sorted_records2 = sorted(sampleset2.record, key=lambda r: r.energy)
variables2 = sampleset2.variables
for record2 in sorted_records2:
    sample = {x_ij: value for x_ij, value in zip(variables2, record2.sample)}
    energy = record2.energy
    grid = grid_solutions(sample)
    regions = [
        *region_lists(grid),
        *row_regions(N),
        *column_regions(N),
    ]
    if energy < -260:
        print('Energy: {} \nSatisfies Constraints: {}'.format(energy, confirm_solution_regions(sample_star, grid, nstars)))
        sample_to_plot(solution, sample)